In [1]:
# Import packages

import York2004 as yk
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import LoadSaveFunctions as lsf

In [ ]:
# Load Dataset

data = lsf.load_excel_file()

print(data.head(5))

In [ ]:
# Compute OLS regression

mod = sm.OLS(data['d18Ow'],sm.add_constant(data['d18Op']))
res = mod.fit()
predictions = res.get_prediction()
prediction_summary = predictions.summary_frame(alpha=0.05)
slope_intercept = res.params
std_err = res.bse
if slope_intercept.iloc[0] < 0:
    label_ols = 'y={:1.2f}±{:1.2f}x{:1.2f}±{:1.2f}'.format(slope_intercept.iloc[1], std_err.iloc[1], slope_intercept.iloc[0], std_err.iloc[0])
else:
    label_ols = 'y={:1.2f}±{:1.2f}x+{:1.2f}±{:1.2f}'.format(slope_intercept.iloc[1], std_err[1], slope_intercept.iloc[0], std_err[0])
print("OLS regression: " + label_ols)

# Compute York et al. 2004 regression

[a_bivar, b_bivar, S, cov, sigma_a, sigma_b] = yk.bivariate_fit(data['d18Op'],data['d18Ow'],data['errx'], data['erry'])

if a_bivar < 0:
    label_york = 'y={:1.2f}±{:1.2f}x{:1.2f}±{:1.2f}'.format(b_bivar, sigma_b, a_bivar, sigma_a)
else:
    label_york = 'y={:1.2f}±{:1.2f}x+{:1.2f}±{:1.2f}'.format(b_bivar, sigma_b, a_bivar, sigma_a)
print("York regression: " + label_york)

In [ ]:
# Plot dataset, OLS and York regressions

fig, ax = plt.subplots()
ax.plot(data['d18Op'], data['d18Ow'], 'bo')
ax.errorbar(data['d18Op'],data['d18Ow'],yerr=data['erry'],xerr=data['errx'], fmt='o')
ax.plot(data['d18Op'], prediction_summary['mean'], 'b-', lw=2, label='OLS: ' + label_ols)
xlim = np.array([min(data['d18Op']), max(data['d18Op'])])
ylim = np.array([min(data['d18Ow']), max(data['d18Ow'])])
ax.plot(xlim, b_bivar*xlim + a_bivar,  'r-', label='York et al, 2004: ' + label_york)
ax.legend()

In [ ]:
# Export Figure in SVG format

lsf.save_figure_as_svg(fig)
